In [1]:
import sqlalchemy as db
from sqlalchemy import create_engine, inspect, MetaData, distinct, func

In [2]:
engine = db.create_engine('sqlite:///C:\\Users\\peter\\Documents\\Projekte\\Zähler\\tc_all_new.db', echo = False)

In [3]:
conn = engine.connect()

#### So komme ich an die Tabellen in der DB!

In [4]:
inspector = inspect(engine)

In [5]:
tab = inspector.get_table_names()
tab = tab[:-1]

#### Und so die Spaltennamen und Typen in einer Tabelle!

In [6]:
columns_table = inspector.get_columns('tcdata_CH_0084_01') 


for c in columns_table :
    print(c['name'], c['type'])

time TEXT
vfr11 REAL
s12 REAL
vfr21 REAL
s22 REAL
vfr1 REAL
s2 REAL


#### Es ist gar nicht so einfach, in SQLalchemy auf Tabellen einer bestehenden DB zuzugreifen. Bei Verwendung von des ORM kann "automate" zur Anwendung kommen. Ohne ORM geht es wie folgt:

In [7]:
metadata = db.MetaData(bind=engine)

In [8]:
table = db.Table('tcdata_CH_0287_02', metadata, autoload = True)

#### Voila, mit der so definierten Tabelle kann ich die Abfragen starten!

#### Wobei wir später durch alle Tabellen loopen werden

## Zunächst: Was will ich alles machen auf der DB?


<ol>
    <li>Voranalyse DB</li> 
        <ul>
         <li>Analyse, was die einzelnen Spalten liefern >> Was wird berücksichtigt?</li>  
         <li>Stichprobe: Check Werte gegen Tool Stundenten </li> 
        </ul>
    <li>Prep DB</li> 
         <ul>
         <li>Ausreisser definieren und raus? >> Oder erst in pandas? </li>
         <li>Geschwindigkeiten runden !!!</li>
         <li>Zeit ggf. auf Lokal umstellen >> Prüfen, was mit SQLalchemy geht </li>     
        </ul>
     <li>DB Auswertungen für Grafiken</li> 
         <ul>
         <li>vfr nach Stunde bzwl nach Wochentag/Ende </li>
         <li>Was tun, wenn ich pro 5min a) keinen bzw. b) mehrere Datenpunkte habe </li>
         <li>speed nach vfr !!!</li>
         <li>Staus </li> 
         <li>Die Spuren im Vergleich </li>   
        </ul>
</ol>

#### Denke, die folgenden Funktionen für das Runden der Geschwindigkeit werden nicht benötigt, da ich das Runden in Pandas machen werde.

func.round(n, x)

from sqlalchemy import update
upd = update(tablename)
val = upd.values({"column_name":"value"})
cond = val.where(tablename.c.column_name == value)

In [9]:
q = db.select(func.count(distinct(table.c.time)), func.count(table.c.time))
result = conn.execute(q)
row = result.fetchone()
row

(67200, 67798)

#### 1. Voranalyse DB

In [10]:
ListLength = []
ListLengthDis = []
ListNull = []

In [11]:
for t in tab:
    table = db.Table(t, metadata, autoload = True)

    ListLength.append(t[10:])
    q = db.select(func.count(table.c.time))
    result = conn.execute(q)
    row = result.fetchone()
    ListLength.append(row) 
    
    ListLengthDis.append(t[10:])
    q = db.select(func.count(distinct(table.c.time)))
    result = conn.execute(q)
    row = result.fetchone()
    ListLengthDis.append(row)    
    
    q = db.select(func.count(table.c.time)).where(table.c.vfr11 == None)
    ListNull.append(t[10:])
    result = conn.execute(q)
    row = result.fetchone()
    ListNull.append(row) 

In [12]:
print(ListLength)

['0020_01', (289397,), '0020_02', (289355,), '0020_03', (289323,), '0020_04', (289366,), '0020_05', (289341,), '0020_06', (289326,), '0020_07', (289379,), '0020_08', (289397,), '0084_01', (289183,), '0084_02', (289183,), '0084_03', (289183,), '0084_04', (289183,), '0150_01', (288074,), '0150_02', (288022,), '0240_01', (289383,), '0240_02', (289397,), '0240_03', (289329,), '0240_04', (289344,), '0240_05', (289333,), '0240_06', (289326,), '0240_07', (289375,), '0240_08', (289397,), '0243_01', (289247,), '0243_02', (288957,), '0243_03', (289163,), '0243_04', (289214,), '0274_01', (288447,), '0274_02', (288233,), '0274_03', (288413,), '0274_04', (288249,), '0274_05', (288050,), '0274_06', (70651,), '0282_01', (289358,), '0282_02', (289358,), '0282_03', (289360,), '0282_04', (289358,), '0282_05', (289358,), '0282_06', (289358,), '0286_01', (16282,), '0286_02', (16282,), '0286_03', (16282,), '0286_04', (16282,), '0286_05', (16282,), '0286_06', (179,), '0286_07', (16282,), '0286_08', (16282,)

In [13]:
print(ListLengthDis)

['0020_01', (288799,), '0020_02', (288757,), '0020_03', (288725,), '0020_04', (288768,), '0020_05', (288743,), '0020_06', (288728,), '0020_07', (288781,), '0020_08', (288799,), '0084_01', (288585,), '0084_02', (288585,), '0084_03', (288585,), '0084_04', (288585,), '0150_01', (287476,), '0150_02', (287424,), '0240_01', (288785,), '0240_02', (288799,), '0240_03', (288731,), '0240_04', (288746,), '0240_05', (288735,), '0240_06', (288728,), '0240_07', (288777,), '0240_08', (288799,), '0243_01', (288649,), '0243_02', (288359,), '0243_03', (288565,), '0243_04', (288616,), '0274_01', (287849,), '0274_02', (287635,), '0274_03', (287815,), '0274_04', (287651,), '0274_05', (287452,), '0274_06', (70053,), '0282_01', (288760,), '0282_02', (288760,), '0282_03', (288762,), '0282_04', (288760,), '0282_05', (288760,), '0282_06', (288760,), '0286_01', (16282,), '0286_02', (16282,), '0286_03', (16282,), '0286_04', (16282,), '0286_05', (16282,), '0286_06', (179,), '0286_07', (16282,), '0286_08', (16282,)

In [14]:
print(ListNull)

['0020_01', (29,), '0020_02', (29,), '0020_03', (29,), '0020_04', (29,), '0020_05', (29,), '0020_06', (29,), '0020_07', (29,), '0020_08', (29,), '0084_01', (6,), '0084_02', (6,), '0084_03', (6,), '0084_04', (6,), '0150_01', (276,), '0150_02', (274,), '0240_01', (29,), '0240_02', (29,), '0240_03', (29,), '0240_04', (29,), '0240_05', (29,), '0240_06', (29,), '0240_07', (29,), '0240_08', (29,), '0243_01', (169,), '0243_02', (264,), '0243_03', (197,), '0243_04', (182,), '0274_01', (114,), '0274_02', (175,), '0274_03', (112,), '0274_04', (159,), '0274_05', (241,), '0274_06', (188,), '0282_01', (62,), '0282_02', (62,), '0282_03', (62,), '0282_04', (62,), '0282_05', (62,), '0282_06', (62,), '0286_01', (0,), '0286_02', (0,), '0286_03', (0,), '0286_04', (0,), '0286_05', (0,), '0286_06', (0,), '0286_07', (0,), '0286_08', (0,), '0287_01', (67798,), '0287_02', (67798,), '0287_03', (67798,), '0287_04', (67798,), '0528_01', (82,), '0528_02', (70,), '0528_03', (121,), '0528_04', (84,), '0529_01', (39

In [15]:
ListTyp0Number = []
ListTyp1Number = []
ListTyp2Number = []
ListSpeedHigh = []

In [16]:
for t in tab:
    table = db.Table(t, metadata, autoload = True)
    
    
    q = db.select(func.count(table.c.time)).where(table.c.vfr11 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListTyp0Number.append(row) 
    q = db.select(func.count(table.c.time)).where(table.c.vfr21 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListTyp1Number.append(row) 
    q = db.select(func.count(table.c.time)).where(table.c.vfr1 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListTyp2Number.append(row)  
    
    ListSpeedHigh.append(t[7:])
    q = db.select(func.count(table.c.time)).where(table.c.s12 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 
    q = db.select(func.max(table.c.s12))
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 
    
    q = db.select(func.count(table.c.time)).where(table.c.s22 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 
    q = db.select(func.max(table.c.s22))
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 
    
    q = db.select(func.count(table.c.time)).where(table.c.s2 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 
    q = db.select(func.max(table.c.s2))
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 


In [17]:
print("Typ0 :", ListTyp0Number, "\n\n")
print("Typ1 :", ListTyp1Number, "\n\n")
print("Typ2 :", ListTyp2Number)

Typ0 : [(594,), (284946,), (278640,), (173527,), (259813,), (282331,), (281931,), (953,), (268759,), (239268,), (256046,), (273538,), (251639,), (234932,), (254689,), (242665,), (283765,), (254884,), (265324,), (284729,), (276287,), (506,), (280004,), (247651,), (252974,), (280876,), (284240,), (283039,), (257172,), (251645,), (282000,), (69899,), (195502,), (257774,), (192750,), (233508,), (278383,), (269292,), (37,), (16039,), (16186,), (15088,), (15620,), (179,), (16010,), (13801,), (0,), (0,), (0,), (0,), (243457,), (173194,), (172565,), (255642,), (136754,), (176703,), (151463,), (147626,), (247150,), (253048,), (253210,), (264594,), (252888,), (150217,), (204073,), (245656,), (147440,), (62174,), (214478,), (228409,), (176253,), (246426,), (245906,), (241067,), (199962,)] 


Typ1 : [(25,), (186801,), (55211,), (184,), (272,), (49084,), (182964,), (127,), (136128,), (924,), (2955,), (136591,), (129253,), (134495,), (75385,), (26476,), (178775,), (7398,), (9938,), (179261,), (80599

In [18]:
print(ListSpeedHigh)

['CH_0020_01', (594,), (144.0,), (25,), (91.0,), (0,), (0.0,), 'CH_0020_02', (284946,), (144.0,), (186801,), (142.0,), (0,), (0.0,), 'CH_0020_03', (278640,), (144.0,), (55211,), (144.0,), (0,), (0.0,), 'CH_0020_04', (173527,), (144.0,), (184,), (144.0,), (0,), (0.0,), 'CH_0020_05', (259813,), (144.0,), (272,), (118.0,), (0,), (0.0,), 'CH_0020_06', (282331,), (144.0,), (49084,), (118.0,), (0,), (0.0,), 'CH_0020_07', (281931,), (144.0,), (182964,), (119.0,), (0,), (0.0,), 'CH_0020_08', (953,), (144.0,), (127,), (88.0,), (0,), (0.0,), 'CH_0084_01', (268757,), (144.0,), (136128,), (113.0,), (0,), (0.0,), 'CH_0084_02', (239258,), (144.0,), (924,), (118.0,), (0,), (0.0,), 'CH_0084_03', (256045,), (144.0,), (2955,), (144.0,), (0,), (0.0,), 'CH_0084_04', (273537,), (144.0,), (136591,), (144.0,), (0,), (0.0,), 'CH_0150_01', (251639,), (210.0,), (129253,), (180.0,), (0,), (0.0,), 'CH_0150_02', (234932,), (126.0,), (134495,), (194.0,), (0,), (0.0,), 'CH_0240_01', (254689,), (144.0,), (75385,), (1